# Water stress impact factors 

Exiobase categories:
- All that match this regex: 'Water Consumption Blue.*'

LC-Impact stressors
- Water stress

In [1]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc
import json

# load arguments
with open("../arguments.json", "r") as f:
    arguments = json.load(f)

In [2]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path=arguments["exio_11_path"])
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path=arguments["exio_19_path"])

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [3]:
# aggregate all blue water consumption related drivers
groups = exio3_11.satellite.get_index(as_dict=True, grouping_pattern={"Water Consumption Blue.*": "Water Consumption Blue -– Total"})

exio3_11.satellite_agg = exio3_11.satellite.copy(new_name="Aggregated blue water consumption accounts")

for df_name, df in zip(exio3_11.satellite_agg.get_DataFrame(data=False, with_unit=True, with_population=False),
exio3_11.satellite_agg.get_DataFrame(data=True, with_unit=True, with_population=False)):
    if df_name == "unit":
        exio3_11.satellite_agg.__dict__[df_name] = df.groupby(groups).apply(lambda x: " & ".join(x.unit.unique()))
    else:
        exio3_11.satellite_agg.__dict__[df_name] = df.groupby(groups).sum()

In [4]:
# diagonalize total blue water consumption and calculate blue water consumption accounts
# optimized for speed
diag_extension = exio3_11.satellite_agg.diag_stressor(("Water Consumption Blue -– Total"))

# First, make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

# calculate S (direct emission multipliers)
if diag_extension.S is None:
    print("Calculating S")
    diag_extension.S = pymrio.calc_S(diag_extension.F, exio3_11.x)

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

diag_extension.D_cba, _, _, _ = pymrio.calc_accounts(diag_extension.S, exio3_11.L, Y_agg)
diag_extension.D_cba

L matrix loaded from pickle
Calculating S


/var/folders/bg/8zzl9cj57fnd569zy539k5h40000gn/T/ipykernel_17527/2743855222.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()


region                                                              AT  \
sector                                                      Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              1.028594e-12   
       Cereal grains nec                                  4.692548e-09   
       Vegetables, fruit, nuts                            1.536466e-09   
       Oil seeds                                          4.273123e-10   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       0.000000e+00   
       Recreational, cultural and sporting services (92)  0.000000e+00   
       Other services (93)                                0.000000e+00   
       Private households with employed persons (95)      0.000000e+00   
       Extra-territorial organizations and bodies         0.000000e+00   

region                                                              \
sector                                                       Wheat   
region sector                                                        
AT     Paddy rice                                         0.000000   
       Wheat                                              0.000098   
       Cereal grains nec                                  0.000005   
       Vegetables, fruit, nuts                            0.000007   
       Oil seeds                                          0.000001   
...                                                            ...   
WM     Membership organisation services n.e.c. (91)       0.000000   
       Recreational, cultural and sporting services (92)  0.000000   
       Other services (93)                                0.000000   
       Private households with employed persons (95)      0.000000   
       Extra-territorial organizations and bodies         0.000000   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.343137e-08   
       Cereal grains nec                                      8.976673e-01   
       Vegetables, fruit, nuts                                3.867929e-05   
       Oil seeds                                              5.300810e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           0.000000e+00   
       Recreational, cultural and sporting services (92)      0.000000e+00   
       Other services (93)                                    0.000000e+00   
       Private households with employed persons (95)          0.000000e+00   
       Extra-territorial organizations and bodies             0.000000e+00   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        1.163782e-07   
       Cereal grains nec                                            2.970657e-04   
       Vegetables, fruit, nuts                                      1.739077e+01   
       Oil seeds                                                    3.705559e-05   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 0.000000e+00   
       Recreational, cultural and sporting servic

## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [5]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns = {}
for series_name, series in diag_extension.D_cba.items():
    series_sum = series.sum()
    columns[series_name] = series / series_sum

dr_s = pd.DataFrame(columns)
dr_s

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              1.041094e-12   
       Cereal grains nec                                  4.749575e-09   
       Vegetables, fruit, nuts                            1.555138e-09   
       Oil seeds                                          4.325053e-10   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       0.000000e+00   
       Recreational, cultural and sporting services (92)  0.000000e+00   
       Other services (93)                                0.000000e+00   
       Private households with employed persons (95)      0.000000e+00   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              1.901168e-05   
       Cereal grains nec                                  1.004692e-06   
       Vegetables, fruit, nuts                            1.282151e-06   
       Oil seeds                                          2.078883e-07   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       0.000000e+00   
       Recreational, cultural and sporting services (92)  0.000000e+00   
       Other services (93)                                0.000000e+00   
       Private households with employed persons (95)      0.000000e+00   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.139423e-09   
       Cereal grains nec                                      7.615176e-02   
       Vegetables, fruit, nuts                                3.281278e-06   
       Oil seeds                                              4.496833e-07   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           0.000000e+00   
       Recreational, cultural and sporting services (92)      0.000000e+00   
       Other services (93)                                    0.000000e+00   
       Private households with employed persons (95)          0.000000e+00   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        4.938578e-10   
       Cereal grains nec                                            1.260616e-06   
       Vegetables, fruit, nuts                                      7.379877e-02   
       Oil seeds                                                    1.572476e-07   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 0.000000e+00   
       Recreational, cultural and sporting services (92)     

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [6]:
# harmonize regions 

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    # custom mappings for countries that pycountry does not recognize
    # these should cover all the countries in the LCI data if country has alpha-2 code
    # these mappings were extracted manually
    extra_mappings = {
        "Turkey": "TR",
        "Russia": "RU",
        "Bahamas, The": "BS",
        "Bonaire": "BQ",
        "Byelarus": "BY",
        "Brunei": "BN",
        "Cape Verde": "CV",
        "Cocos Islands": "CC",
        "Congo DRC": "CD",
        "China, Hong Kong Special Administrative Region": "HK",
        "Curacao": "CW",
        "Democratic Republic of the Congo": "CD",
        "Falkland Islands": "FK",
        "Falkland Islands (Islas Malvinas)": "FK",
        "Gambia, The": "GM",
        "Gaza Strip": "PS",
        "Heard Island & McDonald Islands": "HM",
        "Ivory Coast": "CI",
        "Macedonia": "MK",
        "The Former Yugoslav Republic of Macedonia": "MK",
        "Macau": "MO",
        "Man, Isle of": "IM",
        "Micronesia": "FM",
        "Myanmar (Burma)": "MM",
        "Netherlands Antilles": "AN",
        "Palestinian Territory": "PS",
        "Pacific Islands (Palau)": "PW",
        "Pitcairn Islands": "PN",
        "Reunion": "RE",
        "Saba": "BQ",
        "Saint Eustatius": "BQ",
        "Saint Helena": "SH",
        "Saint Martin": "MF",
        "Sint Maarten": "SX",
        "South Georgia and the South Sandwich Is": "GS",
        "South Georgia": "GS",
        "St. Helena": "SH",
        "Saint Barthelemy": "BL",
        "Saint Kitts and Nevis": "KN",
        "St. Kitts and Nevis": "KN",
        "St. Lucia": "LC",
        "St. Pierre and Miquelon": "PM",
        "Sao Tomo and Principe": "ST",
        "St. Vincent and the Grenadines": "VC",
        "Svalbard": "SJ",
        "Jan Mayen": "SJ",
        "Swaziland": "SZ",
        "US Virgin Islands": "VI",
        "Virgin Islands": "VG",
        "Western Samoa": "WS",
        "West Bank": "PS",
    }
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Alpha-2 country code does not exist for ", name)
            return None

In [9]:
# load and prepare lc-impact data
# TODO: should we use 'all effects' or 'certain effects'?
lci = pd.read_excel(arguments["lc_impact_path"] + "/12-water consumption/CFs_water_consumption_ecosystems_20180831.xlsx",
                    sheet_name="CF per countries",
                    dtype={1: float, 2: float},
                    skiprows=2)
lci = lci[["Country", "CF all effects  [PDF·yr/m3]"]]
# Add country codes to the LCI data and drop rows without country codes
lci["Country_Code"] = lci["Country"].apply(get_country_code)
lci.dropna(subset=["Country_Code"], inplace=True)
lci

,Country,CF all effects [PDF·yr/m3],Country_Code
0,Afghanistan,1.567708e-14,AF
1,Albania,3.239912e-15,AL
2,Algeria,7.703947e-14,DZ
3,Angola,4.276781e-15,AO
4,Argentina,2.634408e-15,AR
...,...,...,...
180,Western Sahara,2.596501e-15,EH
181,Western Samoa,0.000000e+00,WS
182,Yemen,5.872605e-14,YE
183,Zambia,2.942037e-15,ZM


In [ ]:
# add regional averages for regions that are not in LCI data

def get_missing_from_lci(exio_regions, lci):
    """
    Get the regions that are in exiobase but not in lci data.
    """
    missing = []
    for region in exio_regions:
        if region not in lci["Country_Code"].tolist():
            missing.append(region)
    return missing

def augment_water(lci_water):
    # malta is missing from lc-impact, add malta as new row with country code MT and EU averages
    cf_all_eu = 9.02E-15
    row = pd.DataFrame({
        "Country": ["Malta"],
        "CF all effects  [PDF·yr/m3]": [cf_all_eu],
        "Country_Code": ["MT"],
    })
    lci_water = pd.concat([lci_water, row], ignore_index=True)
    return lci_water

exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
if len(get_missing_from_lci(exio_regions_without_row, lci)) > 0:
    print("Missing from LCI:", get_missing_from_lci(exio_regions_without_row, lci))
    lci = augment_water(lci)
    assert len(get_missing_from_lci(exio_regions_without_row, lci)) == 0, "There are still missing regions after augmentation"

Missing from LCI: ['MT']


In [11]:
# COMMON
# harmonize regions in LCI data

def get_row_regions(lci_country_codes, exio_country_codes):
    """
    Get the country codes from lci countries that don't exist in exiobase i.e. rest of the world countries.
    """
    row_regions = []
    for country in lci_country_codes:
        if country not in exio_country_codes:
            row_regions.append(country)
    
    # find duplicates in the list
    duplicates = []
    unique_regions = []
    seen_once = set()
    for item in row_regions:
        if item not in seen_once:
            unique_regions.append(item)
            seen_once.add(item)
        else:
            duplicates.append(item)
    if duplicates:
        print("Duplicates found in row regions:", duplicates)
    return unique_regions

row_countries = get_row_regions(lci["Country_Code"].tolist(), exio_regions)
print("Row regions:", row_countries)

# Load region mappings from arguments
row_eu_countries = arguments["row_region_mappings"]["row_eu"]
row_asia_pacific_countries = arguments["row_region_mappings"]["row_asia_pacific"]
row_african_countries = arguments["row_region_mappings"]["row_africa"]
row_american_countries = arguments["row_region_mappings"]["row_america"]
row_middle_eastern_countries = arguments["row_region_mappings"]["row_middle_east"]

Row regions: ['AF', 'AL', 'DZ', 'AO', 'AR', 'AM', 'AZ', 'BS', 'BD', 'BY', 'BZ', 'BJ', 'BT', 'BO', 'BA', 'BW', 'BN', 'BF', 'BI', 'KH', 'CM', 'CV', 'CF', 'TD', 'CL', 'CO', 'KM', 'CG', 'CD', 'CK', 'CR', 'CU', 'DJ', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'ET', 'FK', 'FO', 'FJ', 'GF', 'PF', 'GA', 'GM', 'GE', 'GH', 'GL', 'GP', 'GT', 'GN', 'GW', 'GY', 'HT', 'HN', 'IS', 'IR', 'IQ', 'IL', 'CI', 'JM', 'JO', 'KZ', 'KE', 'KI', 'KW', 'KG', 'LA', 'LB', 'LS', 'LR', 'LY', 'MK', 'MG', 'MW', 'MY', 'ML', 'MR', 'MU', 'MD', 'MN', 'ME', 'MA', 'MZ', 'MM', 'NA', 'NP', 'NC', 'NZ', 'NI', 'NE', 'NG', 'KP', 'OM', 'PK', 'PA', 'PG', 'PY', 'PE', 'PH', 'PR', 'QA', 'RE', 'RW', 'ST', 'SA', 'SN', 'RS', 'SL', 'SB', 'SO', 'LK', 'VC', 'SD', 'SR', 'SJ', 'SZ', 'SY', 'TJ', 'TZ', 'TH', 'TG', 'TT', 'TN', 'TM', 'UG', 'UA', 'AE', 'UY', 'UZ', 'VU', 'VE', 'VN', 'VG', 'PS', 'EH', 'WS', 'YE', 'ZM', 'ZW']


In [17]:
# augment dr_s to create dr_u

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl = dr_s.loc["WL"].copy()
wl = wl / len(row_american_countries)

we = dr_s.loc["WE"].copy()
we = we / len(row_eu_countries)

wa = dr_s.loc["WA"].copy()
wa = wa / len(row_asia_pacific_countries)

wf = dr_s.loc["WF"].copy()
wf = wf / len(row_african_countries)

wm = dr_s.loc["WM"].copy()
wm = wm / len(row_middle_eastern_countries)

dr_u = dr_s.copy()
dr_u = dr_u.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in row_countries:
    if region in row_eu_countries:
        country_to_region[region] = we
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa
    elif region in row_african_countries:
        country_to_region[region] = wf
    elif region in row_american_countries:
        country_to_region[region] = wl
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in row_countries:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u = pd.concat([dr_u, combined_data])
# drop row region columns
dr_u = dr_u.drop(columns=row_regions.keys(), axis=1, level=0)
dr_u

AT  \
                                                        Paddy rice   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              1.041094e-12   
   Cereal grains nec                                  4.749575e-09   
   Vegetables, fruit, nuts                            1.555138e-09   
   Oil seeds                                          4.325053e-10   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       0.000000e+00   
   Recreational, cultural and sporting services (92)  0.000000e+00   
   Other services (93)                                0.000000e+00   
   Private households with employed persons (95)      0.000000e+00   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                    \
                                                             Wheat   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              1.901168e-05   
   Cereal grains nec                                  1.004692e-06   
   Vegetables, fruit, nuts                            1.282151e-06   
   Oil seeds                                          2.078883e-07   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       0.000000e+00   
   Recreational, cultural and sporting services (92)  0.000000e+00   
   Other services (93)                                0.000000e+00   
   Private households with employed persons (95)      0.000000e+00   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.139423e-09   
   Cereal grains nec                                      7.615176e-02   
   Vegetables, fruit, nuts                                3.281278e-06   
   Oil seeds                                              4.496833e-07   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           0.000000e+00   
   Recreational, cultural and sporting services (92)      0.000000e+00   
   Other services (93)                                    0.000000e+00   
   Private households with employed persons (95)          0.000000e+00   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        4.938578e-10   
   Cereal grains nec                                            1.260616e-06   
   Vegetables, fruit, nuts                                      7.379877e-02   
   Oil seeds                                                    1.572476e-07   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 0.000000e+00   
   Recreational, cultural and sporting services (92)            0.000000e+00   
   Other services (93)                                          0.000000e+00   
   Private households with employed persons (95)                0.000000e+00   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                    

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [18]:
# aggregate all blue water consumption related drivers
groups = exio3_19.satellite.get_index(as_dict=True, grouping_pattern={"Water Consumption Blue.*": "Water Consumption Blue -– Total"})

exio3_19.satellite_agg = exio3_19.satellite.copy(new_name="Aggregated blue water consumption accounts")

for df_name, df in zip(exio3_19.satellite_agg.get_DataFrame(data=False, with_unit=True, with_population=False),
exio3_19.satellite_agg.get_DataFrame(data=True, with_unit=True, with_population=False)):
    if df_name == "unit":
        exio3_19.satellite_agg.__dict__[df_name] = df.groupby(groups).apply(lambda x: " & ".join(x.unit.unique()))
    else:
        exio3_19.satellite_agg.__dict__[df_name] = df.groupby(groups).sum()

In [19]:
# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
dr_f = dr_u.copy()
satellite_cleaned = exio3_19.satellite_agg.M.drop(columns=row_regions.keys(), axis=1, level=0)
total = satellite_cleaned.loc["Water Consumption Blue -– Total"]
scalars = total.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u.shape)

# multiply each column of dr_u by the respective column value
dr_f = dr_f * scalars
dr_f

8800
(37200, 8800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  1.848925e-07   
   Cereal grains nec                                        0.0  9.770835e-09   
   Vegetables, fruit, nuts                                  0.0  1.246918e-08   
   Oil seeds                                                0.0  2.021756e-09   
...                                                         ...           ...   
ZW Membership organisation services n.e.c. (91)             0.0  0.000000e+00   
   Recreational, cultural and sporting services (92)        0.0  0.000000e+00   
   Other services (93)                                      0.0  0.000000e+00   
   Private households with employed persons (95)            0.0  0.000000e+00   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  7.296232e-12   
   Cereal grains nec                                      4.876337e-04   
   Vegetables, fruit, nuts                                2.101149e-08   
   Oil seeds                                              2.879523e-09   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           0.000000e+00   
   Recreational, cultural and sporting services (92)      0.000000e+00   
   Other services (93)                                    0.000000e+00   
   Private households with employed persons (95)          0.000000e+00   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        1.354132e-11   
   Cereal grains nec                                            3.456543e-08   
   Vegetables, fruit, nuts                                      2.023524e-03   
   Oil seeds                                                    4.311647e-09   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 0.000000e+00   
   Recreational, cultural and sporting services (92)            0.000000e+00   
   Other services (93)                                          0.000000e+00   
   Private households with employed persons (95)                0.000000e+00   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              2.306812e-12   
   Cereal grains nec                                  4.181442e-09   
   Vegetables, fruit, nuts                            6.116777e-09   
   Oil seeds                                          1.606631e-05   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       0.000000e+00   
   Recreational, cultural and sporting services (92)  0.000000e+00   
   Other services (93)                                0.000000e+00   
   Private households with employed persons (95)      0.000000e+00   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [ ]:
# WATER
lci_copy = lci.copy()
lci_copy.set_index("Country_Code", inplace=True)

# Ensure lci index is unique before reindexing
lci_copy = lci_copy[~lci_copy.index.duplicated(keep='first')]

# sort rows on lci in same order as dr_f_w
lci_reindexed = lci_copy.reindex(dr_f.index.get_level_values(0).unique())

cf = lci_reindexed["CF all effects  [PDF·yr/m3]"].to_numpy()
cf = np.repeat(cf, 200)
cf = np.tile(cf, (dr_f.shape[1], 1)).T

pdf = dr_f * cf
pdf_total = pdf.sum()

KeyError: 'CF all effects [PDF·yr/m3]'

In [34]:
pd.DataFrame(pdf_total).to_csv("csv/pdf-water.csv", index=True)